In [ ]:
#数据读取以及xy提取
import pandas as pd
import matplotlib as plt
#样例数据读取
df = pd.read_excel('')
#根据共线性矩阵,保留与房价相关性最高的日间人口,将夜间人口和夜间20-29岁人口进行比例处理
def age_percent(row):
    if row['夜间人口'] == 0:
        return 0
    else:
        return row['夜间20-29岁人口']/row['夜间人口']
df['per_a20-29'] = df.apply(age_percent,axis=1)
df = df.drop(columns=['夜间人口','夜间20-29岁人口'])
#制作标签变量
price_median = df['房价']median()
df['is_high'] = df['房价'].map(lambda x: True if x >= price_median else False)
print(df['is_high'].value_counts())
#数据集基本情况查看
print(df.shape)
print(df.dtypes)
#划分数据集
x = df[['日间人口','夜间人口','夜间20-29岁人口','','','']]
print(x.shape)
y = df[['is_high']]
print(y.shape)
print(df.isnull().sum())

In [ ]:
#构建分类模型的交叉验证策略
from sklearn.model_selection import StratifiedKFold
k = 5
kf = StratifiedKFold(n_splits=k,shuffle=True)
kf.get_n_splits(x, y)
print(kf)


In [ ]:
from sklearn.preprocessing import PowerTransformer, StandardScaler,PolynomialFeatures
from sklearn.pipeline import Pipeline
#pip install xgboost

xgb_model = xgb_XGBClassifier(objective='binary:hinge',
                              nthread=16,
                              booster = 'gbtree',
                              n_estimators=500,
                              learning_rate=0.05,
                              max_depth =9,
                              subsample=0.8,
                              colsample_byutree=0.8)

pipe_clf = Pipeline([
    ('sc',StandardScaler()),
    ('power_trans',PowerTransformer()),
    ('xgb_clf',xgb_model)
    ])
print(pipe_clf)


xgboost模型参数讲解
通用参数：
booster：默认gbtree
    - gbtree:基于树的模型
    - gbliner：线性模型
nthread：最大线程数

objective：任务类型
    - 回归任务：reg：squarederror
    - 二元分类任务：
        - binary：logistic（输出概率）
        - binary：hinge（输出分类结果）
    - 其他任务类型详解：https://xgboost.readthedocs.io/en/latest/parameter.html#learning-task-parameters

    训练参数 以tree booster为例子：
    n_estimators：自学习器数量
    learning rate“训练步长
    max_depth:树最大深度
    max_leaf_nodes:树最大节点或叶子数量
    subsample:控制每棵树的训练样本比例
    solsumple_bytree：控制每棵树，训练特征比例
    lambda:L2正则系数
    alpha：L1正则系数
    其他参数详解：https://xgboost.readthedocs.io/en/latest/python/python_api.html#model_xgboost.sklearn


In [ ]:
import warnings
from sklearn.metrics import accuracy_score,precision_score,recall_score,roc_auc_score
warnings.filterwarnings('ignore')
acc = []
precision = []
recall = []
auc = []
for train_index, test_index in kf.split(x,y):
    x_traincv, x_testcv = x.loc[train_index],x.loc[test_index]
    y_traincv, y_testcv = y.loc[train_index], y.loc[test_index]
    pipe_clf.fit(x_traincv, y_traincv)  #训练
    y_predictcv = pipe_clf.predict(x_testcv) #预测
    k_acc = accuracy_score(y_testcv,y_predictcv)
    print(f'accuracy score is: {k_acc}')
    acc.append(k_acc)
    k_precision = precision_score(y_testcv,y_predictcv)
    print(f'precision score is: {k_precision}')
    precision.append(k_precision)
    k_recall = recall_score(y_testcv,y_predictcv)
    print(f'recall score is: {k_recall}')
    recall.append(k_recall)
    k_auc = roc_auc_score(y_testcv,y_predictcv)
    print(f'auc: {k_auc}')
    auc.append(k_auc)
import numpy as np
print(f'---------------------------------')
print(f'average accuracy score is :{np.array(acc).mean()}')
print(f'average precison  is :{np.array(precison).mean()}')
print(f'average recall score is :{np.array(recall).mean()}')
print(f'average auc is :{np.array(auc).mean()}')